In [1]:
import pandas as pd

data = pd.read_csv('./matched.csv')
data.head(2)

,Unnamed: 0,주문번호,수령방법,결제방법,항목,추가선택,드레싱,서비스타입,주문금액,배달팁,결제금액,주문시각,접수시각,배달시각,주소,요청사항,배달요청사항,요금요청,최소시간차,실제배달료
0,3705,B0QJ006GZL,배달,바로결제,"훈제연어 샐러드, 목살 샐러드",NaN,"렌치, 오리엔탈",배민,13300,3000,16300,2020-08-13 11:03:20,2020-08-13 11:03:40,2020-08-13 12:03:40,전북 전주시 덕진구 금암2동 ****,NaN,조심히 안전하게 와주세요 :),[25],25.0,3500.0
1,3706,B0QJ00707X,배달,바로결제,"이너프 샐러드, 이너프 샐러드, 이너프 샐러드",NaN,렌치,배민,12900,3000,15900,2020-08-13 11:04:03,2020-08-13 11:04:57,2020-08-13 12:04:57,전북 전주시 덕진구 덕진동 ****,(수저포크X),조심히 안전하게 와주세요 :),[24],24.0,3700.0


In [2]:
baemin_delivery_tip = dict()
actual_delivery_tip = dict()
food_price = dict()
count = dict()

for idx, row in data.iterrows():
    for address in row['주소'].split(): 
        if '동' in address:
            if address in count:
                baemin_delivery_tip[address] += row['배달팁']
                actual_delivery_tip[address] += row['실제배달료']
                food_price[address] += row['주문금액']
                count[address] += 1
            else:
                baemin_delivery_tip[address] = row['배달팁']
                actual_delivery_tip[address] = row['실제배달료']
                food_price[address] = row['주문금액']
                count[address] = 1
                
baemin_delivery_tip = dict((k, round(baemin_delivery_tip[k]/count[k])) for k in baemin_delivery_tip)
actual_delivery_tip = dict((k, round(actual_delivery_tip[k]/count[k])) for k in actual_delivery_tip)
food_price = dict((k, round(food_price[k]/count[k])) for k in food_price)
baemin_actual_diff = dict((k, baemin_delivery_tip[k]-actual_delivery_tip[k]) for k in baemin_delivery_tip)

print(baemin_delivery_tip)
print(actual_delivery_tip)
print(food_price)
print(baemin_actual_diff)

{'금암2동': 3000, '덕진동': 3000, '금암1동': 3000, '송천2동': 3500, '인후2동': 3500, '중화산2동': 4500, '송천1동': 4000, '서신동': 3500}
{'금암2동': 3400, '덕진동': 3367, '금암1동': 3300, '송천2동': 3700, '인후2동': 3500, '중화산2동': 4600, '송천1동': 5100, '서신동': 3300}
{'금암2동': 12800, '덕진동': 11700, '금암1동': 13040, '송천2동': 18650, '인후2동': 11800, '중화산2동': 22400, '송천1동': 10200, '서신동': 20600}
{'금암2동': -400, '덕진동': -367, '금암1동': -300, '송천2동': -200, '인후2동': 0, '중화산2동': -100, '송천1동': -1100, '서신동': 200}


In [3]:
columns = list(count.keys()) + ['총합']
df = pd.DataFrame(0, index=['배민배달팁', '실제배달료', '차액', '음식값'], columns=columns)
df

,금암2동,덕진동,금암1동,송천2동,인후2동,중화산2동,송천1동,서신동,총합
배민배달팁,0,0,0,0,0,0,0,0,0
실제배달료,0,0,0,0,0,0,0,0,0
차액,0,0,0,0,0,0,0,0,0
음식값,0,0,0,0,0,0,0,0,0


In [4]:
col_num = len(df.columns)-1
df.loc['배민배달팁'] = baemin_delivery_tip
df.loc['배민배달팁', '총합'] = round(df.loc['배민배달팁'].sum()/col_num)

df.loc['실제배달료'] = actual_delivery_tip
df.loc['실제배달료', '총합'] = round(df.loc['실제배달료'].sum()/col_num)

df.loc['음식값'] = food_price
df.loc['음식값', '총합'] = round(df.loc['음식값'].sum()/col_num)

df.loc['차액'] = baemin_actual_diff
df.loc['차액', '총합'] = round(df.loc['차액'].sum()/col_num) 

In [5]:
df

,금암2동,덕진동,금암1동,송천2동,인후2동,중화산2동,송천1동,서신동,총합
배민배달팁,3000.0,3000.0,3000.0,3500.0,3500.0,4500.0,4000.0,3500.0,3500.0
실제배달료,3400.0,3367.0,3300.0,3700.0,3500.0,4600.0,5100.0,3300.0,3783.0
차액,-400.0,-367.0,-300.0,-200.0,0.0,-100.0,-1100.0,200.0,-283.0
음식값,12800.0,11700.0,13040.0,18650.0,11800.0,22400.0,10200.0,20600.0,15149.0
